In [1]:
import geopandas as gpd
import pandas as pd
from geopandas.tools import sjoin_nearest
import os

In [3]:
# === Пути к файлам (относительно notebooks/) ===
base_path = "../data/processed"

sport1_path = os.path.join(base_path, "sport_629.geojson")
sport2_path = os.path.join(base_path, "sport_893.geojson")
stops_path = os.path.join(base_path, "transport_stops.geojson")
output_path = os.path.join(base_path, "sport_with_stops.geojson")

In [4]:
sport1 = gpd.read_file(sport1_path)
sport2 = gpd.read_file(sport2_path)
sports = pd.concat([sport1, sport2]).reset_index(drop=True)

stops = gpd.read_file(stops_path)

Skipping field Services: unsupported OGR type: 5


In [5]:
# === Приведение к одной проекции (метры) ===
sports = sports.to_crs("EPSG:3857")
stops = stops.to_crs("EPSG:3857")

In [6]:
# === Поиск ближайшей остановки к каждому спортобъекту ===
nearest_stops = gpd.sjoin_nearest(sports, stops, how="left", distance_col="dist_to_stop")

In [7]:
# === Очистка и переименование столбцов ===
nearest_stops = nearest_stops[["FullName", "geometry", "global_id_left", "global_id_right", "dist_to_stop"]]
nearest_stops.rename(columns={
    "global_id_left": "sport_global_id",
    "global_id_right": "stop_global_id"
}, inplace=True)

In [ ]:
# === Сохранение результата ===
output_path = "../data/processed/sport_with_stops.geojson"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
nearest_stops.to_file(output_path, driver="GeoJSON")

print(f"[INFO] Сопоставлено: {len(nearest_stops)} спортобъектов с ближайшими остановками")
print(f"[INFO] Сохранено в: {output_path}")


[INFO] Сопоставлено: 13181 спортобъектов с ближайшими остановками
[INFO] Сохранено в: data/processed/sport_with_stops.geojson
